In [1]:
import numpy as np
from scipy.io import loadmat
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
import pickle

# data = loadmat('uft_flight07.mat', variable_names=('lowT_av', 'upT_av', 'lwc1V_av'))

# with open("data.pickle", "wb") as f:
#     pickle.dump(data, f)

with open("data.pickle", "rb") as f:
    data = pickle.load(f)

In [4]:
LOW = 275000
HIGH = 300000

Y1 = data['lowT_av'].squeeze()
Y2 = data['upT_av'].squeeze()
LWC = data['lwc1V_av']
X = np.arange(Y1.shape[0]) / 100.

X_cut = X[LOW:HIGH]
Y1_cut = Y1[LOW:HIGH]
Y2_cut = Y2[LOW:HIGH]

In [5]:
def baseline_heuristic(data):
    # Decent threshold: 0.01
    return data.max(axis=1) - data.min(axis=1)


def base_mean_heuristic(data):
    # Decent threshold: 0.007
    return data.max(axis=1) - data.mean(axis=1)


def base_median_heuristic(data):
    return data.max(axis=1) - np.median(data, axis=1)


def std_small_heuristic(data):
    # Decent threshold: 5
    return (data.max(axis=1) - data.min(axis=1)) / data.std(axis=1)


def std_wide_heuristic(data):
    N = data.shape[1]
    mid = int((N - 1) / 2)
    data = data / data.std(axis=1, keepdims=True)
    return (data[:,mid-2:mid+3].max(axis=1) - data[:,mid-2:mid+3].min(axis=1))


def std_new_heuristic(data):
    N = data.shape[1]
    mid = (N - 1) / 2
    data = data / data.std(axis=1, keepdims=True)
    return (data[:,mid:N+1] - data[:,N-2:N+3].min(axis=1))


def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)


def detect_jumps(data, size=5, threshold=.01, func=baseline_heuristic):
    assert size % 2 # Only odd sizes, for now
    pad = size // 2
    
    strided_data = rolling_window(data, size)
    #diff = strided_data.max(axis=1) - strided_data.min(axis=1)
    result = func(strided_data)
    result = np.hstack((np.zeros(pad), result, np.zeros(pad)))
    return result > threshold


In [6]:
jumps = detect_jumps(Y1_cut, 7, .018, base_median_heuristic)
np.where(jumps)[0].shape

(229,)

In [7]:
plt.rcParams['figure.figsize'] = (18,4)

WIDTH = 0.5

#jumps = detect_jumps(Y1_cut, 9, 3.75, std_wide_heuristic)
#jumps = detect_jumps(Y1_cut, 5, .005, base_mean_heuristic)

highlight = np.where(jumps)[0] / 100.
highlight += LOW / 100.


plt.vlines(highlight, 13.8, 14.6, alpha=.25, linewidth=2.5, colors='r')
plt.plot(X_cut, Y1_cut, linewidth=WIDTH)
#plt.plot(X_cut, Y2_cut, linewidth=WIDTH)
#plt.plot(X[LOW:HIGH], 20*LWC[LOW:HIGH])
plt.show()

In [8]:
plt.plot(X_cut, Y1_cut, linewidth=WIDTH)

plt.show()

In [9]:
WINDOW = 21
PAD = WINDOW // 2

rolled = rolling_window(Y1_cut, WINDOW).mean(1)
rolled = np.hstack((np.ones((PAD,)) * rolled[:PAD].mean(), rolled, np.ones((PAD,)) * rolled[-PAD:].mean()))

plt.plot(X_cut, Y1_cut - rolled, linewidth=0.5)
#plt.plot(X_cut, Y1_cut);plt.plot(X_cut, rolled)

plt.show()

## Global normalization - probably redundant

In [181]:
Y1_cut.mean()

14.302825324832844

In [33]:
Y1_norm = (Y1 - Y1.mean()) / Y1.std()

WINDOW = 21
PAD = WINDOW // 2

#rolled = rolling_window(Y1_norm, WINDOW).mean(1) # median?
rolled = np.median(rolling_window(Y1_norm, WINDOW), axis=1)
rolled = np.hstack((np.ones((PAD,)) * rolled[:PAD].mean(), rolled, np.ones((PAD,)) * rolled[-PAD:].mean()))

#plt.plot(X, Y1_norm - rolled, linewidth=0.5)
plt.plot(X, Y1_norm);plt.plot(X, rolled)

plt.show()

KeyboardInterrupt: 

# To do:

0. Estimate the optimal threshold value, maybe via a purely statistical analysis (i.e. fraction of all points that are marked as jumps)

0. Estimate the optimal window (other than 5 - maybe)

0. Try to make it more robust to excessive noise (divide the diff term by std?)

0. Substract linear baseline

0. More sophisticated algorithm?

# Ideas:

Use the smoothed-out line as a baseline or something.

Measure the maximum distance between baseline and actual series

On a slightly larger window, fourier and defourier (filtering)? (or other denoising)

Change mean to median, std to median absolute deviation

# General idea:

Get a smoothed-out version of the series, measure distance between it and original

Przefiltrować sygnał, np. running median

Butterworth filter

Jakaś pochodna 

Przebudować pipeline:

sygnał -> filtr -> heura -> threshold

Narysować PDF składowych wiatru dla obszarów z zaburzeniami i bez zaburzeń, może 2-wymiarowy (histogram)